# Importing the necessary libraries

In [1]:
import pandas as pd
import datetime

## Reading the data

In [2]:
df = pd.read_excel('Analyst Assignment .xlsx',sheet_name= 'June Sale')
df.head()

,Sale Order Code,Facility,Channel Name,SKU Name,Uniware SKU Code,Seller SKU Code,MRP,Total Price,Selling Price,Order Date,...,CGST Rate,IGST Rate,SGST Rate,UTGST Rate,CESS Rate,Tax %,Tax Value,Voucher Code,Discount,Shipping Charges
0,SLP2305749392,dazzlecollectionz,SNAPDEAL_PRITA,Pritas Raani Haar,DC-NL-10187,DC-NL-10187,2000.0,629.0,629.0,2019-06-01 00:01:00,...,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0
1,11600000000000000,FLIPKART_LITE,FLIPKART_LITE,Bahubali earrings,EAR-PR-50008,DC--EAR-50169,1400.0,469.0,429.0,2019-06-01 00:30:00,...,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,40.0,40
2,11600000000000000,FLIPKART_LITE,FLIPKART_LITE,Bahubali earrings,EAR-PR-50008,DC--EAR-50169,1400.0,469.0,429.0,2019-06-01 00:30:00,...,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,40.0,40
3,11600000000000000,FLIPKART_LITE,FLIPKART_LITE,Priyaasi's Partywear Kundan Single Finger Hath...,HPHL-PR-10001,DC-HPHL-10001,1100.0,429.0,429.0,2019-06-01 00:30:00,...,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0
4,11600000000000000,FLIPKART_LITE,FLIPKART_LITE,Priyaasi's Partywear Kundan Single Finger Hath...,HPHL-PR-10001,DC-HPHL-10001,1100.0,429.0,429.0,2019-06-01 00:30:00,...,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0


### The purpose of this notebook is to get a feel of the data and understand what should be the move forward

In [3]:
df.iloc[0]

Sale Order Code                 SLP2305749392
Facility                    dazzlecollectionz
Channel Name                   SNAPDEAL_PRITA
SKU Name                    Pritas Raani Haar
Uniware SKU Code                  DC-NL-10187
Seller SKU Code                   DC-NL-10187
MRP                                      2000
Total Price                               629
Selling Price                             629
Order Date                2019-06-01 00:01:00
Sale Order Status                    COMPLETE
Sale Order Item Status             DISPATCHED
GST Tax Type Code                 DEFAULT_GST
CGST                                        0
IGST                                    18.32
SGST                                        0
UTGST                                       0
CESS                                        0
CGST Rate                                   0
IGST Rate                                   3
SGST Rate                                   0
UTGST Rate                        

#### Considering our goal of calculating the inventory restock requirement , we do not require any information regarding the cost and taxes. 

In [4]:
# Only Picking out the necessary information
df = df[['Sale Order Code','Uniware SKU Code','Order Date','Sale Order Status']]

In [5]:
set(df['Sale Order Status'])

{'CANCELLED', 'COMPLETE', 'PROCESSING'}

In [6]:
df['Sale Order Status'].value_counts()

COMPLETE      17375
CANCELLED       700
PROCESSING        4
Name: Sale Order Status, dtype: int64

#### Filtering out data for only completed and processing

In [7]:
df = df[df['Sale Order Status']!='CANCELLED']
len(df)

17379

# Next let us analyse whether the day of the weak i.e Mon-Sun affects the no. of orders.



### This is extremely important as Jun 2019 saw 5 weekends and we should be careful while predicting the inventory requirements

In [8]:
df['dayofweek'] = df['Order Date'].apply(lambda x : x.weekday())
df['Date'] = df['Order Date'].apply(lambda x : x.date())
df[['Date','dayofweek']].drop_duplicates().groupby('dayofweek').agg('count').rename(columns={'Date' : 'Noofdays'})

,Noofdays
dayofweek,
0,4
1,4
2,4
3,4
4,4
5,5
6,5


## 0 - Sun, 6 - Sat

In [9]:
df[['Date','dayofweek','Sale Order Status']].groupby(['dayofweek','Date']).agg('count')

Sale Order Status
dayofweek Date                         
0         2019-06-03                243
          2019-06-10                184
          2019-06-17                257
          2019-06-24               1101
1         2019-06-04                226
          2019-06-11                256
          2019-06-18                182
          2019-06-25               1779
2         2019-06-05                176
          2019-06-12                225
          2019-06-19                421
          2019-06-26                331
3         2019-06-06                217
          2019-06-13               1492
          2019-06-20                450
          2019-06-27                325
4         2019-06-07                151
          2019-06-14                250
          2019-06-21               4118
          2019-06-28                357
5         2019-06-01                196
          2019-06-08                258
          2019-06-15                179
          2019-06-22               1679
          2019-06-29                337
6         2019-06-02                280
          2019-06-09                180
          2019-06-16                242
          2019-06-23                972
          2019-06-30                315

In [10]:
a = df[['Date','Sale Order Status']].groupby(['Date']).agg('count')
a.plot()

## FROM THE ABOVE : We can infer that there has been an abnormal spike in the no. of orders between 21-23 June

## Spike of this nature can generally be attributed to Advertisements, marketing campaigns, heavy discount offers etc.

In [11]:
d = pd.merge(df[['dayofweek','Date']].groupby('dayofweek').agg('count'),df[['Date','dayofweek']].drop_duplicates().groupby('dayofweek').agg('count'),on='dayofweek')

In [12]:
d = d.rename(columns = {'Date_x' : 'Count of Orders', 'Date_y' : 'No of days'})

In [13]:
d['avg'] = d['Count of Orders']/d['No of days']

In [14]:
d

,Count of Orders,No of days,avg
dayofweek,,,
0,1785,4,446.25
1,2443,4,610.75
2,1153,4,288.25
3,2484,4,621.00
4,4876,4,1219.00
5,2649,5,529.80
6,1989,5,397.80


### While the above Data suggests an average to below average response on the weekend. The difference can largely be attributed to the sudden spike observed above between 21-23 june. Hence, the attempt to understand the effect of the day of the week on the number of orders is inconclusive

## As we can't really pin-point and accomodate for the reason for heavy increase in number of orders on these dates, we shall go forward considering exponential smoothening of our values to carry out our inventory requirement predictions.